In [1]:
import seaborn as sns 
df=sns.load_dataset('tips')

In [2]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
df.time

0      Dinner
1      Dinner
2      Dinner
3      Dinner
4      Dinner
        ...  
239    Dinner
240    Dinner
241    Dinner
242    Dinner
243    Dinner
Name: time, Length: 244, dtype: category
Categories (2, object): ['Lunch', 'Dinner']

In [4]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()

In [5]:
df['time']= encoder.fit_transform(df.time)

In [6]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [7]:
df.time.unique()

array([0, 1])

In [8]:
# dividing dependent and independent
X= df.drop(labels=['time'],axis=1)
y=df.time

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer   ## handling missing value
from sklearn.preprocessing import StandardScaler  # scaling of datar
from sklearn.preprocessing import OneHotEncoder    ## categorical to numerical cols
from sklearn.compose import ColumnTransformer


In [11]:
Categorical_cols= ['sex','smoker','day']
Numerical_cols= ['total_bill','tip','size']

In [12]:
# automate the feature Engineering

num_pipleline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')),
        ('Scalar',StandardScaler())
    ]
)

cat_pipeline = Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('OneHotEncoder',OneHotEncoder())
    ]
)

In [13]:
# joining the pipeline

preprocessor = ColumnTransformer([
    ('num_pipleline',num_pipleline,Numerical_cols),
    ('cat_pipeline',cat_pipeline,Categorical_cols)
]
)

In [14]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [15]:
X_train

array([[-0.79306155, -0.2580329 , -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.46322744, -0.74211442, -0.61214068, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.80730659,  0.6399734 , -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.65383098, -1.46472887, -0.61214068, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.64749986,  0.32426806, -0.61214068, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.75289699, -0.41237773,  0.45363997, ...,  1.        ,
         0.        ,  0.        ]])

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [17]:
models = {
    'Random forest' : RandomForestClassifier(),
    'logistic regression' : LogisticRegression(),
    'decison tree'  : DecisionTreeClassifier()
}
len(models)

3

In [18]:
def evaluate_model(X_train, X_test, y_train,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        model_score = accuracy_score(y_test,y_pred)
        report[list(models.keys())[i]] = model_score
    return report

In [19]:
evaluate_model(X_train, X_test, y_train, y_test,models)

{'Random forest': 0.9591836734693877,
 'logistic regression': 1.0,
 'decison tree': 0.9387755102040817}

In [20]:
# hyperparameter tuning 
classifire = RandomForestClassifier()


In [21]:
parameter = {
    'max_depth':[2,3,56,None],
    'n_estimators':[100,200,300,400,50],
    'criterion':['gini','entropy']

}

In [22]:
from sklearn.model_selection import GridSearchCV

final_model = GridSearchCV(classifire,param_grid=parameter,scoring='accuracy',cv=10,verbose=3)
final_model.fit(X_train,y_train)

Fitting 10 folds for each of 40 candidates, totalling 400 fits
[CV 1/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.950 total time=   0.0s
[CV 2/10] END criterion=gini, max_depth=2, n_estimators=100;, score=1.000 total time=   0.0s
[CV 3/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.950 total time=   0.0s
[CV 4/10] END criterion=gini, max_depth=2, n_estimators=100;, score=1.000 total time=   0.0s
[CV 5/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.950 total time=   0.0s
[CV 6/10] END criterion=gini, max_depth=2, n_estimators=100;, score=1.000 total time=   0.0s
[CV 7/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.895 total time=   0.0s
[CV 8/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.947 total time=   0.0s
[CV 9/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.947 total time=   0.0s
[CV 10/10] END criterion=gini, max_depth=2, n_estimators=100;, score=0.947 total time=   0.0s
[CV 1/

GridSearchCV(cv=10, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 56, None],
                         'n_estimators': [100, 200, 300, 400, 50]},
             scoring='accuracy', verbose=3)

In [23]:
y_pred = final_model.predict(X_test)

In [24]:
accuracy_score(y_test,y_pred)

1.0